In [ ]:
# 사용방법 - 복사하여 주석만 제거하여 사용
import sys
from pathlib import Path

# 현재 노트북(.ipynb)이 src/ 안에 있으니 상위 폴더(프로젝트 루트)를 추가
BASE_DIR    = Path().resolve().parent    # Notebook이 src/ 안이라면 .parent
config_path = BASE_DIR / 'config' / 'data_paths.yaml'
proj_root = Path().resolve().parent
sys.path.insert(0, str(proj_root))


# 이제 바로 import
from scripts.data_loader import load_data

# 테스트
df_pm10 = load_data('pm10')
# or 
# df = load_data('pm10', section='raw')
df_pm10

,구분,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,세종특별자치시,수원,...,함양,남해,산청,의령,창녕,합천,도평균.7,제주,서귀포,도평균.8
0,2006.01 월,65,52,60,73,72,56,37,NaN,74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44,49,NaN
1,2006.02 월,67,55,61,72,65,55,44,NaN,76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,45,NaN
2,2006.03 월,81,82,82,83,83,69,81,NaN,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66,56,NaN
3,2006.04 월,107,93,87,111,82,82,91,NaN,129,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,64,NaN
4,2006.05 월,63,61,52,69,64,51,55,NaN,71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49,50,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,2024.06 월,31,26,27,30,23,27,34,28.0,33,...,22.0,23.0,25.0,23.0,26.0,23.0,24.0,22,23,23.0
222,2024.07 월,20,15,16,23,14,15,22,17.0,22,...,15.0,14.0,16.0,18.0,18.0,16.0,16.0,17,21,19.0
223,2024.08 월,24,19,17,27,19,19,23,23.0,26,...,19.0,18.0,19.0,21.0,21.0,19.0,20.0,23,22,23.0
224,2024.09 월,16,15,16,18,16,14,20,17.0,17,...,16.0,13.0,17.0,18.0,18.0,16.0,16.0,21,22,21.0


In [ ]:
import pandas as pd 
from datetime import datetime

df = load_data('pm10', section='raw')
df_long = df.melt(id_vars='구분', var_name='region', value_name='pm10')
df_long['pm10'] = pd.to_numeric(df_long['pm10'], errors='coerce')
missing_counts = df_long.groupby('region')['pm10'].apply(lambda x: x.isna().sum())
valid_regions = missing_counts[missing_counts < 10].index
df_filtered = df_long[df_long['region'].isin(valid_regions)]

avg_pm10_filtered = (
    df_filtered.groupby('region')['pm10']
    .mean()
    .sort_values(ascending=False)
    .head(30)
    .reset_index()
)

avg_pm10_filtered.columns = ['지역', '평균 PM10 농도']
avg_pm10_filtered.insert(0, '순위', range(1, len(avg_pm10_filtered) + 1))

display("결측치 10개 미만 지역 상위 30 미세먼지 평균 농도 도시", avg_pm10_filtered)

today = datetime.now().strftime('%Y%m%d')
filename = f"pm10_top30_{today}_v1.csv"

out_dir  = BASE_DIR / 'data' / 'processed'
out_path = out_dir / filename

out_dir.mkdir(parents=True, exist_ok=True)
# avg_pm10_filtered.to_csv(out_path, index=False, encoding='utf-8-sig') # 실제 저장 코드 

print(f"Top30 PM10 결과를 저장했습니다: {out_path}")

'결측치 10개 미만 지역 상위 30 미세먼지 평균 농도 도시'

,순위,지역,평균 PM10 농도
0,1,평택,58.491150
1,2,이천,56.164444
2,3,포천,55.814480
3,4,양주,55.351111
4,5,동두천,54.863014
5,6,김포,54.309735
6,7,화성,54.269912
7,8,부천,53.876106
8,9,고양,52.871681
9,10,시흥,52.725664


Top30 PM10 결과를 저장했습니다: C:\team_project\dust\data\processed\pm10_top30_20250529_v1.csv
